In [1]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [2]:
#| export
from decimal import Decimal, getcontext
from typing import Union
from urllib.request import urlretrieve, URLError
import xmltodict
from zipfile import ZipFile


import pandas as pd
import pyodbc
from rich.console import Console
from pyarrow import ArrowInvalid, ArrowTypeError
from unidecode import unidecode
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import test_eq


from anateldb.constants import *
from anateldb.format import parse_bw, format_types, input_coordenates

getcontext().prec = 5

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [3]:
#| export
def connect_db(server: str = 'ANATELBDRO05', # Servidor do Banco de Dados
               database: str = 'SITARWEB', # Nome do Banco de Dados
               trusted_conn: str = 'yes', # Conexão Segura: yes | no
               mult_results: bool = True, # Múltiplos Resultados
              )->pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )

In [4]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM,STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [5]:
#| exporti
def _parse_estações(row: dict)->dict:
    """Given a row in a MongoDB ( a dict of dicts ), it travels some keys and return a subset dict"""
    
    d = {k.replace('@', '').lower():row[k] for k in ("@SiglaServico", "@id", "@state",
        "@entidade",
        "@fistel",
        "@cnpj",
        "@municipio",
        "@uf")}
    entidade = row.get('entidade', {})
    d.update({k.replace('@', '').lower():entidade[k] for k in ('@num_servico', '@habilitacao_DataValFreq')})
    administrativo = row.get('administrativo', {})
    d['numero_estacao'] = administrativo.get('@numero_estacao')
    estacao = row.get('estacao_principal', {})
    d.update({k.replace('@', '').lower():estacao[k] for k in ('@latitude', '@longitude')})
    return d

In [6]:
#| exporti
def _read_estações(path: Union[str, Path]) -> pd.DataFrame:
    """Read the zipped xml file `Estações.zip` from MOSAICO and returns a dataframe"""
    
    with ZipFile(path) as myzip:
        with myzip.open('estacao_rd.xml') as myfile:
            estacoes = xmltodict.parse(myfile.read())
            
    assert 'estacao_rd' in estacoes, "The xml file inside estacoes.zip is not in the expected format"
    assert 'row' in estacoes['estacao_rd'], "The xml file inside estacoes.zip is not in the expected format"
    
    df = pd.DataFrame(L(estacoes['estacao_rd']['row']).map(_parse_estações))
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    df = df.loc[:, COL_ESTACOES]
    df.columns = NEW_ESTACOES    
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    return df

In [7]:
#| exporti
def _parse_pb(row: dict)->dict:
    """Given a row in the MongoDB file canais.zip ( a dict of dicts ), it travels some keys and return a subset dict"""
    return {unidecode(k).lower().replace("@", ""): v  for k,v in row.items()}

In [8]:
#| exporti
def _read_plano_basico(path: Union[str, Path]) -> pd.DataFrame:
    """Read the zipped xml file `Plano_Básico.zip` from MOSAICO and returns a dataframe"""    
    df = L()
    with ZipFile(path) as myzip:
        with myzip.open('plano_basicoTVFM.xml') as myfile:
            pbtvfm = xmltodict.parse(myfile.read())
        with myzip.open('plano_basicoAM.xml') as myfile:
            pbam = xmltodict.parse(myfile.read())
        with myzip.open('secundariosTVFM.xml') as myfile:
            stvfm = xmltodict.parse(myfile.read())
        with myzip.open('secundariosAM.xml') as myfile:
            sam = xmltodict.parse(myfile.read())    
            
    for base in (pbtvfm, stvfm, pbam, sam):
        assert 'plano_basico' in base, "The xml files inside canais.zip is not in the expected format"
        assert 'row' in base['plano_basico'], "The xml file inside canais.zip is not in the expected format"
        df.extend(L(base['plano_basico']['row']).map(_parse_pb))
        
    df = pd.DataFrame(df)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)    
    df.columns = NEW_PB
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    df.loc[:, 'Frequência'] = df.Frequência.str.replace(',', '.')
    for c in df.columns:
        df.loc[df[c] == '', c] = pd.NA
    return df    

In [9]:
#| export
def clean_mosaico(df: pd.DataFrame, # DataFrame com os dados de Estações e Plano_Básico mesclados 
                pasta: Union[str, Path], # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame: # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        df.loc[df[col_x].isna(), col_x] = df.loc[df[col_x].isna(), col_y]
        df.loc[df[col_y].isna(), col_y] = df.loc[df[col_y].isna(), col_x]
        if df[col_x].notna().sum() > df[col_y].notna().sum():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor == "", "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor == "", "Longitude_Estação"
    ]
    df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor.isna(), "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor.isna(), "Longitude_Estação"
    ]
    df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    df.rename(
        columns={
            "Latitude_Transmissor": "Latitude",
            "Longitude_Transmissor": "Longitude",
        },
        inplace=True,
    )

    df = input_coordenates(df, pasta)
    df.loc[:, "Frequência"] = df.Frequência.str.replace(",", ".")
    if freq_nans := df[df.Frequência.isna()].Id.tolist():
        complement_df = scrape_dataframe(freq_nans)
        df.loc[
            df.Frequência.isna(),
            [
                "Status",
                "Entidade",
                "Fistel",
                "Frequência",
                "Classe",
                "Num_Serviço",
                "Município",
                "UF",
            ],
        ] = complement_df.values

    df.loc[:, "Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == "OM", "Frequência"] = df.loc[
        df.Serviço == "OM", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df.drop_duplicates(keep="first").reset_index(drop=True)

## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [10]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = format_types(df, stem)
    df = df.drop_duplicates(keep='first').reset_index(drop=True)
    df = df.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError):
        file.unlink(missing_ok=True)
        try:
            file = Path(f"{folder}/{stem}.fth")
            df.to_feather(file)
        except (ArrowInvalid, ArrowTypeError):
            file.unlink(missing_ok=True)
            try:
                file = Path(f"{folder}/{stem}.xlsx")
                with pd.ExcelWriter(file) as wb:
                    df.to_excel(
                        wb, sheet_name="DataBase", engine="openpyxl", index=False
                    )
            except Exception as e:
                raise ValueError(f"Could not save {stem} to {file}") from e
    return df



In [11]:
# | export
def update_radcom(folder: Union[str, Path], # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(RADCOM, conn)
            return _save_df(df, folder, "radcom")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [12]:
# | export
def update_stel(folder:Union[str, Path], # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="bouncingBall",
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(STEL, conn)
            return _save_df(df, folder, "stel")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [13]:
stel = update_stel(Path.cwd().parent / 'dados')

(  ●   ) Lendo o Banco de Dados do STEL. Processo Lento, aguarde...

In [16]:
stel.to_parquet(Path.cwd().parent / 'dados' / 'stel.parquet.gzip', compression='gzip', index=False)

In [ ]:
# | export
def update_mosaico(folder: Union[str, Path], # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local do Mosaico. É baixado e processado arquivos xml zipados da página pública do Spectrum E"""
    console = Console()
    with console.status(
        "[blue]Baixando e consolidando os dados do Mosaico...", spinner="clock"
    ) as status:  
        try:
            stations, _ = urlretrieve(ESTACOES, f"{folder}/estações.zip")
            pb, _ = urlretrieve(PLANO_BASICO, f"{folder}/canais.zip")
            estações = _read_estações(stations)
            plano_basico = _read_plano_basico(pb)
            df = estações.merge(plano_basico, on="Id", how="left")
            df = clean_mosaico(df, folder)
            return _save_df(df, folder, "mosaico")
        except URLError as e:
            status.console.log("[red]Não foi possível baixar os dados do Mosaico")
            raise ConnectionError from e



In [ ]:
# | export
def update_base(folder: Union[str, Path], # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as três bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    try:
        stel = update_stel(folder).loc[:, TELECOM]
    except ConnectionError:
        from anateldb.reading import read_stel
        stel = read_stel(folder).loc[:, TELECOM]
    try:
        radcom = update_radcom(folder).loc[:, SRD]
    except ConnectionError:
        from anateldb.reading import read_radcom
        radcom = read_radcom(folder).loc[:, SRD]
    try:
        mosaico = update_mosaico(folder).loc[:, RADIODIFUSAO]
    except ConnectionError:
        from anateldb.reading import read_mosaico
        mosaico = read_mosaico(folder).loc[:, RADIODIFUSAO]
    radcom["Num_Serviço"] = "231"
    radcom["Status"] = "RADCOM"
    radcom["Classe_Emissão"] = pd.NA
    radcom["Largura_Emissão"] = BW_MAP["231"]
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Validade_RF"] = pd.NA
    radcom["Fonte"] = "SRD"
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    mosaico["Fonte"] = "MOS"
    mosaico["Classe_Emissão"] = pd.NA
    mosaico["Largura_Emissão"] = mosaico.Num_Serviço.map(BW_MAP)
    rd = (
        pd.concat([mosaico, radcom, stel])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )
    rd = rd.drop_duplicates(keep="first").reset_index(drop=True)
    rd["BW(kHz)"] = rd.Largura_Emissão.astype('string').fillna('-1').apply(parse_bw)
    return _save_df(rd, folder, "base")

In [ ]:
#| hide
folder = Path.cwd().parent / 'dados'

In [ ]:
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# req = Request(ESTACAO)
# try:
#     response = urlopen(req)
# except URLError as e:
#     if hasattr(e, 'reason'):
#         print('We failed to reach a server.')
#         print('Reason: ', e.reason)
#     elif hasattr(e, 'code'):
#         print('The server couldn\'t fulfill the request.')
#         print('Error code: ', e.code)
# else:
#     Path.cwd().joinpath('estações.zip').write_bytes(response.read())